In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_2.csv")

In [3]:
df.head()

,timestamp_local,temp,city_name,country_code,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,Gujrāt,PK,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,Gujrāt,PK,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,Gujrāt,PK,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,Gujrāt,PK,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,Gujrāt,PK,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
data = df[['pm25']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled = scaled.reshape((19706,))

In [8]:
scaled.shape

(19706,)

In [9]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [10]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [11]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [12]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0062 - root_mean_squared_error: 0.0772 - val_loss: 0.0037 - val_root_mean_squared_error: 0.0610 - learning_rate: 0.0010
Epoch 2/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0037 - root_mean_squared_error: 0.0612 - val_loss: 0.0037 - val_root_mean_squared_error: 0.0606 - learning_rate: 0.0010
Epoch 3/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0035 - root_mean_squared_error: 0.0593 - val_loss: 0.0034 - val_root_mean_squared_error: 0.0579 - learning_rate: 0.0010
Epoch 4/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0034 - root_mean_squared_error: 0.0582 - val_loss: 0.0033 - val_root_mean_squared_error: 0.0579 - learning_rate: 0.0010
Epoch 5/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0033 - root_mean_squared_error: 0.0571 - val_loss: 0.0033 - val_root_mean_squared_error: 0.0575 - learning_rate: 0.0010
Epoch 6/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0033 - root_mean_squared_e

In [13]:
forecast = model.predict(X_test)

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [14]:
poll = np.array(df["pm25"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 3.5172045


In [17]:
y_test_true.shape

(3874, 168)

In [18]:
testPredict.shape

(3874, 168)

In [19]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [20]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 0.7603
RMSE for segment 2: 1.1413
RMSE for segment 3: 1.5626
RMSE for segment 4: 1.8698
RMSE for segment 5: 2.1294
RMSE for segment 6: 2.3465
RMSE for segment 7: 2.4991
RMSE for segment 8: 2.6040
RMSE for segment 9: 2.6869
RMSE for segment 10: 2.7366
RMSE for segment 11: 2.7662
RMSE for segment 12: 2.7775
RMSE for segment 13: 2.8197
RMSE for segment 14: 2.8729
RMSE for segment 15: 2.9396
RMSE for segment 16: 2.9450
RMSE for segment 17: 2.9251
RMSE for segment 18: 2.9185
RMSE for segment 19: 2.8994
RMSE for segment 20: 2.8786
RMSE for segment 21: 2.8618
RMSE for segment 22: 2.8618
RMSE for segment 23: 2.8667
RMSE for segment 24: 2.9031
RMSE for segment 25: 2.9743
RMSE for segment 26: 2.9840
RMSE for segment 27: 3.0243
RMSE for segment 28: 3.0787
RMSE for segment 29: 3.1409
RMSE for segment 30: 3.1659
RMSE for segment 31: 3.1815
RMSE for segment 32: 3.2186
RMSE for segment 33: 3.2810
RMSE for segment 34: 3.3024
RMSE for segment 35: 3.2661
RMSE for segment 36: 3.2446
R

In [21]:
forecast

array([[0.08048381, 0.07467578, 0.06992105, ..., 0.08197425, 0.08020122,
        0.07953435],
       [0.10448924, 0.10192487, 0.10453976, ..., 0.1626497 , 0.15221184,
        0.14641044],
       [0.05743064, 0.05653492, 0.05456128, ..., 0.05848379, 0.05706321,
        0.05670088],
       ...,
       [0.15346214, 0.15776126, 0.1708462 , ..., 0.19479358, 0.18172857,
        0.1671418 ],
       [0.45238256, 0.4341074 , 0.36444584, ..., 0.37625173, 0.3804539 ,
        0.36466834],
       [0.04589932, 0.04859518, 0.05199139, ..., 0.04744203, 0.05138961,
        0.0554092 ]], dtype=float32)

In [22]:
testPredict

array([[ 83.83237 ,  83.45889 ,  83.15315 , ...,  83.92821 ,  83.81419 ,
         83.77132 ],
       [ 85.37599 ,  85.21109 ,  85.379234, ...,  89.11588 ,  88.444695,
         88.07165 ],
       [ 82.34998 ,  82.29238 ,  82.165474, ...,  82.4177  ,  82.326355,
         82.303055],
       ...,
       [ 88.52509 ,  88.80154 ,  89.64294 , ...,  91.18283 ,  90.342705,
         89.40473 ],
       [107.74657 , 106.57141 , 102.09197 , ..., 102.85112 , 103.12134 ,
        102.10628 ],
       [ 81.60848 ,  81.78184 ,  82.00022 , ...,  81.70769 ,  81.961525,
         82.21999 ]], dtype=float32)

In [23]:
y_test_true

array([[ 83.99237 ,  84.028305,  84.06532 , ...,  84.86345 ,  84.463844,
         84.06532 ],
       [ 85.29899 ,  85.33492 ,  85.37194 , ...,  88.02111 ,  87.07708 ,
         86.13413 ],
       [ 82.17726 ,  81.77875 ,  81.379135, ...,  83.085365,  83.15723 ,
         83.23018 ],
       ...,
       [ 87.912224,  88.5296  ,  89.14589 , ..., 103.372765,  99.88737 ,
         96.40305 ],
       [105.4786  , 103.48274 , 101.23209 , ..., 100.50692 ,  99.56289 ,
         99.45401 ],
       [ 81.63284 ,  81.99651 ,  82.3591  , ...,  80.72583 ,  81.379135,
         82.03245 ]], dtype=float32)

In [72]:
last_input = X_test[-1]

In [73]:
last_input.shape

(168, 1)

In [74]:
last_input = last_input.reshape(168,)

In [77]:
# predictions = []
# for _ in range(168):
#     # Predict the next value
#     print(last_input)
predicted_value = model.predict(last_input.reshape(1, 168, 1))[0]
predicted_value = predicted_value
predicted_value = predicted_value * stdop + meanop
    # predictions.append(predicted_value)
    
#     # Update the input sequence for the next iteration
#     last_input = np.append(last_input[1:], predicted_value)
# print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


In [78]:
predicted_value

array([ 88.56814 ,  87.0865  ,  85.89468 ,  85.56283 ,  85.12053 ,
        84.926125,  84.82616 ,  84.900276,  85.012024,  86.242004,
        87.66179 ,  89.064674,  93.91494 ,  98.99813 , 103.95776 ,
       104.310104, 104.34928 , 104.4447  , 102.89404 , 101.59718 ,
        99.94553 ,  96.863625,  93.75132 ,  90.66343 ,  89.166595,
        88.023994,  86.89277 ,  86.74036 ,  86.463684,  86.19579 ,
        86.0873  ,  85.94686 ,  85.92076 ,  86.63535 ,  87.637215,
        88.65329 ,  93.195015,  98.06164 , 103.030815, 104.152336,
       104.91652 , 105.34003 , 103.85257 , 102.05101 , 100.2362  ,
        96.88373 ,  93.53139 ,  90.27503 ,  88.84191 ,  87.40513 ,
        86.02289 ,  85.74353 ,  85.2294  ,  85.013855,  84.87958 ,
        85.02116 ,  85.15712 ,  86.54288 ,  88.187225,  89.803764,
        94.41786 ,  99.27209 , 104.060486, 104.77241 , 105.30119 ,
       105.5344  , 103.812996, 101.92653 ,  99.97254 ,  96.67348 ,
        93.36318 ,  90.06612 ,  88.883865,  87.41988 ,  86.257

In [35]:
y_test[-168:].ravel() * stdop + meanop

array([30.597725, 26.346886, 28.059309, 31.56474 , 28.663692, 29.550125,
       30.980501, 25.601479, 27.394485, 29.328516, 29.932901, 28.583109,
       27.172878, 28.139893, 31.504303, 29.932901, 25.601479, 26.346886,
       26.125278, 29.187494, 27.53551 , 28.663692, 30.899918, 27.253462,
       29.630709, 25.682062, 30.013485, 27.918285, 26.649078, 28.059309,
       25.299286, 29.550125, 26.870686, 28.804716, 26.2663  , 28.361502,
       26.2663  , 31.806494, 25.682062, 25.762648, 28.804716, 25.984255,
       28.663692, 26.346886, 26.649078, 28.804716, 24.553877, 26.508055,
       29.187494, 25.823086, 28.885302, 30.456701, 30.235092, 28.744278,
       24.715046, 29.187494, 25.238846, 29.630709, 27.47507 , 30.376118,
       29.711294, 24.553877, 31.423717, 29.106909, 29.026325, 25.460455,
       28.139893, 24.775486, 30.980501, 27.01171 , 27.696678, 24.997093,
       25.460455, 27.777262, 29.106909, 27.8377  , 28.965885, 28.059309,
       29.550125, 29.932901, 28.583109, 30.013485, 

In [24]:
model.save("pm25_1dcnn.keras")